In [4]:
import json, requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import feedparser
# from markdownify import markdownify

In [5]:
nature_url = "https://www.nature.com"
subject_url = nature_url + "/search?article_type=research, reviews, protocols&subject={}&date_range=last_7_days&order=date_desc"

In [3]:
subjects = ["Physics", "Chemistry", "Engineering", "Materials science", 
    "Energy science and technology", "Mathematics and computing", "Astronomy and planetary science", 
    "Business and industry", "Energy and society", "Geography", "Science", "technology and society", 
    "Social policy", "Politics and international relations","Computational biology and bioinformatics", 
    "Chemical biology, Biotechnology", "Information systems and information technology", "Computer science", 
    "Machine learning", "Image processing", "Signal processing"]

subjects = [i.lower().replace(" ","-") for i in subjects]

In [6]:
date_str = "20 March 2023"
date_obj = datetime.strptime(date_str, '%d %B %Y')
date_obj

datetime.datetime(2023, 3, 20, 0, 0)

In [7]:
r = requests.request("GET", subject_url.format("biochemistry"))
soup = BeautifulSoup(r.content, "html.parser")

In [53]:
soup.find("a",{"class":"c-card__link u-link-inherit"})["href"]

'/articles/s41467-023-37281-7'

In [60]:
url = nature_url+soup.find("a",{"class":"c-card__link u-link-inherit"})["href"]
r = requests.request("GET", url)
soup = BeautifulSoup(r.content, "html.parser")

In [63]:
soup.find("div", {"class": "c-article-body main-content"})

In [35]:
def summerize(text : str):
    
    payload = {
    "input" : {
        "text":text,
        "min_length" : 300,
        "max_length" : 500
        }
    }

    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

    r = requests.post("http://localhost:5000/predictions", data=json.dumps(payload), headers=headers).json()
    
    return r["output"][0]["summary_text"]

In [36]:
def translate(text : str):
    
    payload = {
    "input" : {
        "inputs":text
        }
    }

    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

    r = requests.post("http://localhost:5001/predictions", data=json.dumps(payload), headers=headers).json()
    
    return r["output"]

In [8]:
def news_content(url: str) -> str:
    
    r = requests.request("GET", url)
    soup = BeautifulSoup(r.content, "html.parser")
    
    if soup.find("div", {"class": "c-article-body main-content"}):
        
        field = ','.join([i.lower() for i in soup.find("ul",{"class":"c-article-subjects u-clearfix u-hide-print"}).text.strip().split("\n\n\n")])
        
        html_content = soup.find("div",{"class":"c-article-body main-content"})
        soup.find("figcaption").decompose()
        
        content = html_content.text
        img_link = "https:" + soup.find("figure",{"class":"figure"}).img["src"]
        
        return field, content, img_link
        
    return None

In [37]:
def one_week_articles() -> list:

    articles = list()
    
    nature_rss = feedparser.parse("https://www.nature.com/nature.rss")
    
    for item in nature_rss["entries"]:
        
        try:
            
            article_link = item["id"]
        
            article_date = datetime.strptime(item["updated"], '%Y-%m-%d') 

            if (datetime.today() - timedelta(weeks=1)) < article_date < datetime.today():

                _data = news_content(article_link)
                field, _content, img_link = _data

                if _data is not None:

                    article = {}

                    article["title"] = translate(item["title"])
                    article["summary"] = item["summary"]

                    article["field"] = field
                    article["content"] = translate(summerize(_content))
                    article["img_link"] = img_link

                    articles.append(article)
        except Exception as Error:
            
            print(f"Cant not scarpe { article_link } due to - {Error}")

    return articles

In [38]:
data = one_week_articles()

Cant not scarpe https://www.nature.com/articles/d41586-023-00844-1 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/d41586-023-00845-0 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/d41586-023-00843-2 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/d41586-023-00710-0 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/d41586-023-00708-8 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/s41586-023-05950-8 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/s41586-023-05958-0 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/d41586-023-00799-3 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/

In [ ]:
len(data)

In [11]:
data

NameError: name 'data' is not defined

In [15]:
text  = """\nLast October, the pioneering life-sciences journal eLife introduced bold changes to its editorial practice — which some researchers applauded as reimagining the purpose of a scientific journal. From 31 January this year, eLife said, it would publish every paper it sent out for peer review: authors would never again receive a rejection after a negative review. Instead, reviewers’ reports would be published alongside the paper, together with a short editorial assessment of the work’s significance and rigour. Authors could then decide whether to revise their paper to address any comments.The change followed an earlier decision by eLife to require that all submissions be posted as preprints online. The cumulative effect was to turn eLife into a producer of public reviews and assessments about online research. It was “relinquishing the traditional journal role of gatekeeper”, editor-in-chief Michael Eisen explained in a press release, and “promoting the evaluation of scientists based on what, rather than where, they publish”.\n\n\n\n\n\nThe transformation sparked enthusiastic praise — and sharp criticism. Some scientists saw it as a long-overdue move to empower authors. Others, including some of eLife’s academic editors (who are mostly senior researchers), weren’t so happy. They worried it would diminish the prestige of a brand they’d worked hard to build, and some wrote privately to Eisen (in letters seen by Nature) to say they would resign if the plan was fully implemented. Amid the pushback, the journal postponed switching fully to its new process.But the dispute only heightened. On 9 March, 29 eLife editors — including the journal’s former editor-in-chief, Randy Schekman — wrote to Damian Pattinson, executive director of the journal’s non-profit publisher, eLife Sciences Publications in Cambridge, UK, asking that Eisen be replaced “immediately”. They added that they had no confidence in Eisen’s leadership, because he had dismissed their concerns and had not considered compromise positions. One of the journal’s five deputy editors had already stepped down from that leadership position, and “significant numbers” of reviewers and senior editors were “standing ready to resign”, they wrote.Eisen, a Howard Hughes Medical Institute (HHMI) investigator who works at the University of California, Berkeley, fired back publicly online, tweeting on 12 March that academics were “lobbying hard to get me fired”. He later deleted the tweet, but told Nature in an interview that “opposition to eLife’s model is driven fundamentally by powerful scientists not wanting to change a system that has benefited them and which they have sculpted to continue to reward them”. In response, Schekman and other authors stated that Eisen’s comments were “not true and do not reflect our legitimate concerns with the new model at eLife”.Eisen says he thinks the dissent is small in scale. He and Pattinson say they did not dismiss concerns, but consulted on changes over two years with editors. “We see big swathes of enthusiasm among the community,” Pattinson adds.The row highlights disagreements among researchers about the function of journals and peer review — and, potentially, about the future of science publishing. Some eLife editors argue that journals should use review to guide filtering and rejection of papers. But supporters of eLife’s changes see benefit in preventing peer review from serving as a prestige-gathering function, in which, by rejecting most of the manuscripts submitted to them, selective journals become perceived as arbiters of what work matters. “We rely too much on journal titles in judging people’s work,” Eisen says. “If we want to fix a bad system, we do have to break some eggs.”What is a journal’s purpose?When eLife was launched in 2012 with the financial backing of three powerful science funders — the Maryland-based HHMI, the UK Wellcome Trust and Germany’s Max Planck Society — it had the aim of being a non-commercial and academic-edited journal that would rival prestigious titles such as Cell, Nature and Science. (Nature’s news team is editorially independent of the journal’s research team.) Besides being open access, another of its key innovations was a collaborative system of peer review, where referees and a handling editor discuss comments together. The journal attracted dozens of working scientists as editors who triage submissions, with hundreds more scientists as reviewing editors.\n\nOpen-access journal eLife announces ‘preprint first’ publishing model\neLife had its eye on bigger changes, however. In 2021, the journal decided to publish only papers that were already preprints. This meant that delays in reviewing wouldn’t hold up an author from sharing their work. But even before Eisen and Pattinson joined, the journal had run a trial with more than 300 manuscripts to test the idea of ditching rejection after review. Its aim was to simply publish papers with reviews, author responses and editorial ratings. “The peer-review process does not need to end with a binary outcome of acceptance or rejection,” the journal wrote in a 2019 analysis of that work.It was this idea that eLife instituted for all papers last October, with the addition that editors would also append a short summary assessment of the paper — giving readers a quick idea of its quality and significance. “This puts power back in the hands of the authors, who can then publish what they have, instead of having to do ever more experiments to satisfy reviewers,” says Eisen. The journal plans to charge US$2,000 for the process of arranging review on submissions; previously, its open-access publication fee was $3,000.Some eLife editors are fully on board with the new system. “It’s the future, where science is going,” says senior eLife editor Panayiota Poirazi, a neuroscientist at the Institute of Molecular Biology and Biotechnology in Heraklion, Crete. Among the journal’s funders, HHMI says it fully supports the new policy. Wellcome says that it supports eLife’s publishing process, and the Max Planck Society told Nature it was still discussing the issue.But other researchers have been openly critical from the start. In November, 47 editors wrote privately to Eisen asking for a rethink or for more time to experiment — perhaps running the new system alongside the conventional one, or creating a second journal in which to publish papers of less significance. They worried about harm to the journal’s collaborative open-reviewing process, and that the quality of papers on the eLife platform would drop. With no possibility of rejection, some authors might choose to ignore reviewer comments or only superficially address them, they wrote — and that knowledge might discourage reviewers from producing detailed critiques. Responding to these concerns, Eisen and Pattinson say that they haven’t seen such problems so far, although the project is in its early days, and that running two systems would reduce the chances of the new model’s success.Editors also argued that removing rejection-after-review meant more pressure on the gatekeeping step that remains in eLife’s system — the triage point where editors choose whether to send out a paper for review. That step had been “opaque and subject to errors in judgment”, their letter stated, an issue that would become more consequential if later negative reviews could no longer lead to rejection. Editors might react by becoming more conservative and decide not to take a chance on manuscripts from less-well-known authors. But Eisen says that, in the new system, sending a preprint for review shouldn’t communicate anything about its quality or importance: the reviews and editorial assessments do that instead. The guidance that editors should follow when deciding what to send for review is “can you generate high-quality and broadly useful public reviews of this paper?”, he says.\n\nWill the pandemic permanently alter scientific publishing?\nIn some countries, hiring and promotion decisions still rely heavily on journal titles in candidates’ publication lists — something that is unlikely to change quickly, the editors added in their letter. They worried that scientists there would stop sending their manuscripts to eLife. Eisen, however, says that problematic reliance on journal titles will continue until there is an alternative system, such as eLife’s.In a further private letter sent to Eisen in January, 30 editors said they would resign once the new policy was fully implemented.The full scale of the discontent is unclear. Although Eisen and Pattinson say they’ve had broad support, Axel Brunger, a structural biologist at Stanford University in California, who initiated the first letter, says he reached out only to his colleagues in structural biology and neuroscience, and that nearly all agreed to sign up. “The concerns are widespread,” he says.One researcher who signed all three letters is neuroscientist Gary Westbrook at the Vollum Institute at Oregon Health & Science University in Portland. He is a vocal critic of what he sees as the monopoly that commercial journals have in science publishing, and says he signed “because I didn’t think the new policy was realistic”. Far from helping eLife as a non-profit, high-quality alternative, he says, he thinks the model will diminish its impact.Reviewing preprintsThe concept of reviewing preprints is catching on in the life sciences. At least two dozen preprint-refereeing initiatives of various sizes have been launched in the past few years. The largest (apart from eLife itself) is Review Commons, launched in December 2019 by the California-based non-profit organization ASAPbio and EMBO Press. The latter runs five journals and is part of the European Molecular Biology Organization in Heidelberg, Germany. As a review-sharing collaboration between 17 journals from 6 publishers, including eLife, Review Commons uses EMBO Press editors to select referees for submissions. Authors can ask Review Commons to post reviews and any further author responses on a preprint server, or they can submit their paper, with reviews and responses, to any journal. More than 2,000 reviews of 540 articles have been run through this system.The idea of ‘journal agnostic’ reviewing is still at proof-of-principle stage, says Bernd Pulverer, EMBO’s head of scientific publications. But he sees merit in having both peer-reviewed preprints and conventional journals, which, he says, provide “real added value in condensing and stratifying information”.That view is shared by Maria Leptin, president of the European Research Council. “If I want to learn about a new field that is not core to my own, then I want a trustworthy source that filters for general interest,” she says. “eLife now does its filtering upstream, in a non-transparent, unaccountable way.”The triage stage shouldn’t be seen as this kind of filter, says Eisen. “People are used to operating in a world where appearance in a journal tells you about the quality, audience or import of a study. This is precisely what we are trying to change,” he says. He argues that the short editorial summary eLife appends to its articles serve as quality guides for readers. They grade the significance of the findings (landmark, fundamental, important, valuable, useful) and assess the strength of their support (exceptional, compelling, convincing, solid, incomplete, inadequate).More consultation?Endocrinologist Mone Zaidi at Icahn School of Medicine at Mount Sinai, New York City, is one of eLife’s four remaining deputy editors and has been trying to mediate the issue. He admires Eisen’s vision, he says, “but any new, transformative change has to be done in a careful manner, with buy-in from the community”.Together with some of his colleagues, he is trying to persuade Eisen to slow down, to avoid mass resignations and to establish milestones to assess the effects the changes would have on the lives of working scientists. “There has to be consultation and risk-mitigation plans,” he says.The deputy editor who stood down, cell biologist Anna Akhmanova at the University of Utrecht in the Netherlands, shares Zaidi’s view. She says she helped to develop the new system, but stepped down as deputy editor because it was being pushed through too fast. “We need evolution, not revolution — many small, careful steps to try to move the community towards what would be a better publishing system,” she says.Eisen says he has already responded to concerns by extending — for a short time — the deadline for the regular reviewing system. “We expect things to evolve in interesting ways as people start to see the advantages and opportunities of not making publishing decisions.”“eLife is doing a big and interesting experiment, however it works out,” says stem-cell biologist Fiona Watt, a former eLife deputy editor who is now EMBO’s director. “My sense as a scientist is that the publishing landscape is changing again."""

In [17]:
payload = {
    "input" : {
        "text":text,
        "min_length" : 300,
        "max_length" : 500
    }
}

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

r = requests.post("http://localhost:5000/predictions", data=json.dumps(payload), headers=headers)

In [23]:
r.json()["output"][0]["summary_text"]

"Life-sciences journal eLife introduced bold changes to its editorial practice. Some researchers applauded it as reimagining the purpose of a scientific journal. Others, including some of eLife’s academic editors, weren’t so happy. The row highlights disagreements among researchers about the function of journals and peer review. Some eLife editors argue that journals should use review to guide filtering and rejection of papers. But supporters of the changes see benefit in preventing peer review from serving as a prestige-gathering function, in which selective journals become perceived as arbiters of what work matters. The journal postponed switching fully to its new process, but the dispute only heightened. The change followed an earlier decision by eLife to require that all submissions be posted as preprints online. It was “relinquishing the traditional journal role of gatekeeper”, editor-in-chief Michael Eisen explained in a press release, and “promoting the evaluation of scientists 

In [34]:
a = translate(r.json()["output"][0]["summary_text"])
a["output"]

'Tạp chí eLife, một tạp chí khoa học đời sống eLife đã đưa ra những thay đổi táo bạo trong việc biên tập. Một số nhà nghiên cứu khen ngợi nó như là hình dung lại mục đích của một tạp chí khoa học. Những người khác, bao gồm một số biên tập viên học thuật của eLife, không vui vẻ lắm. Hàng này nêu bật những bất đồng giữa các nhà nghiên cứu về chức năng của tạp chí và bình duyệt. Một số biên tập viên eLife cho rằng các tạp chí nên sử dụng đánh giá để hướng dẫn lọc và từ chối bài báo. Nhưng những người ủng hộ những thay đổi này thấy lợi ích trong việc ngăn chặn bình duyệt đóng vai trò như một chức năng thu thập uy tín, trong đó các tạp chí chọn lọc trở nên được coi là những người phân xử về những gì công việc quan trọng. Tạp chí hoãn chuyển đổi hoàn toàn sang quy trình mới, nhưng tranh chấp chỉ tăng lên. Sự thay đổi này theo sau một quyết định trước đó của eLife yêu cầu tất cả các bài dự thi phải được đăng dưới dạng bản in sẵn trực tuyến. Đó là "phục hồi vai trò của người gác cổng truyền th

In [8]:
r = requests.get("https://www.nature.com/articles/d41586-023-00795-7")
soup = BeautifulSoup(r.content, "html.parser")

In [9]:
soup.find("ul",{"class":"c-article-subjects u-clearfix u-hide-print"}).text.strip().split("\n\n\n")

['Careers', 'Physics', 'Nanoscience and technology']

In [42]:
soup.find("figure",{"class":"figure"}).img["src"]

'//media.nature.com/w767/magazine-assets/d41586-023-00795-7/d41586-023-00795-7_24630594.jpg'

In [26]:
_test_ls = [
    "https://www.nature.com/articles/d41586-023-00831-6",
    "https://www.nature.com/articles/d41586-023-00818-3",
    "https://www.nature.com/articles/d41586-023-00816-5"
]

In [100]:
soup.find("img",{"class":"figure__image"})["src"]

'//media.nature.com/lw767/magazine-assets/d41586-023-00831-6/d41586-023-00831-6_24636118.jpg'

In [28]:
lorem = """Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."""

articles = list()
for link in _test_ls:
    _data = article_content(link)
            
    if _data is not None:

        article = {}

        article["title"] = "Sample Title"
        article["summary"] = lorem

        _data = article_content(link)

        field, _content, img_link = _data

        article["field"] = field
        article["content"] = _content
        article["img_link"] = img_link
        
        articles.append(article)

In [29]:
from jinja2 import Environment, FileSystemLoader

In [30]:
articles

[{'title': 'Sample Title',
  'summary': "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.",
  'field': 'Publishing',
  'content': '\nLast October, the pioneering life-sciences journal eLife introduced bold changes to its editorial practice — which some researchers applauded as reimagining the purpose of a scientific journal. From 31 January this year, eLife said, it would publish every paper it sent out for peer review: authors would never again receive a rejection afte

In [53]:
results_filename = "sample.html"
results_template = environment.get_template("index.html")
context = {
    "datetime" : tuple( ((datetime.today() - timedelta(weeks=1)).strftime("%d/%m/%Y"), datetime.today().strftime("%d/%m/%Y")))
    "data": data,
}
with open(results_filename, mode="w", encoding="utf-8") as results:
    results.write(results_template.render(context))
    print(f"... wrote {results_filename}")

... wrote sample.html


In [8]:
datetime.today().strftime("%d/%m/%Y")

'14/03/2023'